In [16]:
import numpy as np
import sympy as sp

In [17]:
h_pq = np.array([[ 4.9348022,  0.       ],
                 [ 0.       , 19.7392088]])

g_pqrs = np.array([[[[1.5, 0. ],
                     [0. , 1. ]],
 
                    [[0. , 1. ],
                     [1. , 0. ]]],
 
 
                   [[[0. , 1. ],
                     [1. , 0. ]],
   
                    [[1. , 0. ],
                     [0. , 1.5]]]])

In [18]:
import numpy as np

# Definir las matrices de Pauli y las matrices Q^+ y Q^-
X = np.array([[0, 1],
              [1, 0]])
Y = np.array([[0, -1j],
              [1j, 0]])
Z = np.array([[1, 0],
              [0, -1]])
I = np.array([[1, 0],
              [0, 1]])

Q_dagger = np.array([[0, 0],
                     [1, 0]])
Q = np.array([[0, 1],
              [0, 0]])

# Definir las matrices de creación y destrucción en función de las matrices de Pauli
def a_dagger_i(i, vector):
    """
    Operador de creación para el qubit i en un sistema de n qubits.
    """
    result = 1
    for j in vector[:i]:
        if j == 1:
            result = -result

    return result * Q_dagger

def a_i(i, vector):
    """
    Operador de destrucción para el qubit i en un sistema de n qubits.
    """
    result = 1
    for j in vector[:i]:
        if j == 1:
            result = -result

    return result * Q


vector = [1, 1, 0, 0]

for i in range(4):
    globals()[f'a_dagger_{i}'] = a_dagger_i(i, vector)
    globals()[f'a_{i}'] = a_i(i, vector)

In [20]:
h_pq = np.array([[ 4.9348022,          0,          0 ,         0],
                 [         0,  4.9348022,          0,          0],
                 [         0,          0, 19.7392088,          0],
                 [         0,          0,          0, 19.7392088]])


vectors = [[1, 1, 0, 0],
           [1, 0, 1, 0],
           [1, 0, 0, 1],
           [0, 1, 1, 0],
           [0, 1, 0, 1],
           [0, 0, 1, 1]]

omega = 1 / np.sqrt(4) * np.array([1, 0, 1, 0, 1, 0, 1, 0])
# omega = 1 / np.sqrt(4) * np.array([0, 1, 0, 1, 0, 1, 0, 1])

H = np.zeros((6, 6))

for i, vector_i in enumerate(vectors):
    for j, vector_j in enumerate(vectors):

        vector_j = list(vector_j)
        vector_j.reverse()

        for p in range(len(h_pq)):
            for q in range(len(h_pq)):
                if h_pq[p][q] != 0:
                    # print(i, j, p, q)
                    
                    I_0 = I
                    I_1 = I
                    I_2 = I
                    I_3 = I
                    # I_4 = I
                    # I_5 = I

                    for n, value in enumerate(vector_j):
                        if value == 1:
                            vector_j_rev = list(vector_j)
                            vector_j_rev.reverse()
                            m = len(vector_j) - 1 - n
                            globals()[f'I_{m}'] = np.dot(globals()[f'I_{m}'], a_i(m, vector_j_rev))
                            

                    globals()[f'I_{p}'] = np.dot(globals()[f'I_{p}'], globals()[f'a_dagger_{p}'])
                    globals()[f'I_{q}'] = np.dot(globals()[f'I_{q}'], globals()[f'a_{q}'])

                    for n, value in enumerate(vector_i):
                        if value == 1:
                            globals()[f'I_{n}'] = np.dot(globals()[f'I_{n}'], a_dagger_i(n, vector_i))
                    

                    n = int(len(omega) / 2)
                    size = 2 * n 
                    matriz_grande = np.zeros((size, size))

                    for t, submatriz in enumerate([I_0, I_1, I_2, I_3]):
                        fila_inicio = 2 * t
                        col_inicio = 2 * t
                        matriz_grande[fila_inicio:fila_inicio+2, col_inicio:col_inicio+2] = submatriz
                    # print(matriz_grande)
                    # print(np.dot(omega, np.dot(matriz_grande, omega.T)) * h_pq[p][q] )
                            
                    H[i][j] += np.dot(omega, np.dot(matriz_grande, omega.T)) * h_pq[p][q]

In [21]:
A = sp.Matrix(H)
A

Matrix([
[ 24.674011, 12.3370055, 12.3370055, 12.3370055, 12.3370055,          0],
[12.3370055,  24.674011, 12.3370055, 12.3370055,          0, 12.3370055],
[12.3370055, 12.3370055,  24.674011,          0, 12.3370055, 12.3370055],
[12.3370055, 12.3370055,          0,  24.674011, 12.3370055, 12.3370055],
[12.3370055,          0, 12.3370055, 12.3370055,  24.674011, 12.3370055],
[         0, 12.3370055, 12.3370055, 12.3370055, 12.3370055,  24.674011]])

In [22]:
autovalores = A.eigenvals()
autovalores

{74.0220330000000: 1,
 24.6740110000000: 3,
 -2.47459564550276e-63: 1,
 3.47273682728984e-63: 1}

In [23]:
eigenvalues, eigenvectors = np.linalg.eigh(H)
print("Autovalores del hamiltoniano:", eigenvalues)

Autovalores del hamiltoniano: [1.39036889e-15 6.15914768e-15 2.46740110e+01 2.46740110e+01
 2.46740110e+01 7.40220330e+01]


In [24]:
# h_pq = np.array([[ 4.9348022]])
h_pq = np.array([[1, 0, 0],
                 [0, 2, 0],
                 [0, 0, 3]])


vectors = [[1, 1, 0],
           [0, 1, 1],
           [1, 0, 1]]
vector = [1, 1, 0]

for i in range(4):
    globals()[f'a_dagger_{i}'] = a_dagger_i(i, vector)
    globals()[f'a_{i}'] = a_i(i, vector)

omega = 1 / np.sqrt(3) * np.array([1, 0, 1, 0, 1, 0])
# omega = np.array([0, 1, 0, 1, 0, 1, 0, 1])

H = np.zeros((3, 3))

for i, vector_i in enumerate(vectors):
    for j, vector_j in enumerate(vectors):

        vector_j = list(vector_j)
        vector_j.reverse()

        for p in range(len(h_pq)):
            for q in range(len(h_pq)):
                if h_pq[p][q] != 0:
                    # print(i, j, p, q)
                    
                    I_0 = I
                    I_1 = I
                    I_2 = I
                    # I_3 = I
                    # I_4 = I
                    # I_5 = I

                    for n, value in enumerate(vector_j):
                        if value == 1:
                            vector_j_rev = list(vector_j)
                            vector_j_rev.reverse()
                            m = len(vector_j) - 1 - n
                            globals()[f'I_{m}'] = np.dot(globals()[f'I_{m}'], a_i(m, vector_j_rev))

                    globals()[f'I_{p}'] = np.dot(globals()[f'I_{p}'], globals()[f'a_dagger_{p}'])
                    globals()[f'I_{q}'] = np.dot(globals()[f'I_{q}'], globals()[f'a_{q}'])

                    for n, value in enumerate(vector_i):
                        if value == 1:
                            globals()[f'I_{n}'] = np.dot(globals()[f'I_{n}'], a_dagger_i(n, vector_i))
                    

                    n = int(len(omega) / 2)
                    size = 2 * n 
                    matriz_grande = np.zeros((size, size))

                    for t, submatriz in enumerate([I_0, I_1, I_2]):
                        fila_inicio = 2 * t
                        col_inicio = 2 * t
                        matriz_grande[fila_inicio:fila_inicio+2, col_inicio:col_inicio+2] = submatriz
                    # print(matriz_grande)
                    
                    
                    if vector_i[p] == 1 and vector_j_rev[q] == 1:
                        print(vector_i)
                        print(vector_j_rev)
                        print(round(np.dot(omega, np.dot(matriz_grande, omega.T)) * h_pq[p][q], 2), i, j, p, q)
                        H[i][j] += np.dot(omega, np.dot(matriz_grande, omega.T)) * h_pq[p][q]
    
A = sp.Matrix(H)
A

[1, 1, 0]
[1, 1, 0]
1.0 0 0 0 0
[1, 1, 0]
[1, 1, 0]
2.0 0 0 1 1
[1, 1, 0]
[0, 1, 1]
-0.67 0 1 1 1
[1, 1, 0]
[1, 0, 1]
0.33 0 2 0 0
[0, 1, 1]
[1, 1, 0]
-0.67 1 0 1 1
[0, 1, 1]
[0, 1, 1]
2.0 1 1 1 1
[0, 1, 1]
[0, 1, 1]
3.0 1 1 2 2
[0, 1, 1]
[1, 0, 1]
1.0 1 2 2 2
[1, 0, 1]
[1, 1, 0]
0.33 2 0 0 0
[1, 0, 1]
[0, 1, 1]
1.0 2 1 2 2
[1, 0, 1]
[1, 0, 1]
1.0 2 2 0 0
[1, 0, 1]
[1, 0, 1]
3.0 2 2 2 2


Matrix([
[               3.0, -0.666666666666667, 0.333333333333333],
[-0.666666666666667,                5.0,               1.0],
[ 0.333333333333333,                1.0,               4.0]])

In [25]:
A.eigenvals()

{5.67934766600044: 1, 2.49670043054909: 1, 3.82395190345048: 1}

In [26]:
omega

array([0.57735027, 0.        , 0.57735027, 0.        , 0.57735027,
       0.        ])